In [39]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans,DBSCAN

from sklearn import preprocessing


pd.set_option('display.max_columns', None)  
df=pd.read_csv('CC GENERAL.csv')

df=df.dropna()
df.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.40,0.000000,0.166667,0.000000,0.083333,0.00,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.00,6442.945483,0.000000,0.000000,0.000000,0.25,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.00,0.000000,1.000000,1.000000,0.000000,0.00,0,12,7500.0,622.066742,627.284787,0.000000,12
4,C10005,817.714335,1.000000,16.00,16.00,0.00,0.000000,0.083333,0.083333,0.000000,0.00,0,1,1200.0,678.334763,244.791237,0.000000,12
5,C10006,1809.828751,1.000000,1333.28,0.00,1333.28,0.000000,0.666667,0.000000,0.583333,0.00,0,8,1800.0,1400.057770,2407.246035,0.000000,12


We can see 19 columns, from "CUST_ID" to "TENURE", which contains both label variable and continuous variables.

We will use column1 to column10 to do clustering since they are meaningful information, and we want to divide them to several groups according to their behaviors.

After then, we can use our domain knowledge to create new business values via "Precision marketing".




In [40]:
cols=df.columns[1:10]


feature_value=preprocessing.scale(df.loc[:,cols])
#feature_value=preprocessing.scale(df.iloc[:,12:-1])

db=DBSCAN(eps=0.3,min_samples=100).fit(feature_value)
cluster_label=db.labels_

In [41]:

for i in np.unique(cluster_label):
    print(' class %d has %d sample '%(i,np.sum(cluster_label==i)))



 class -1 has 7132 sample 
 class 0 has 862 sample 
 class 1 has 454 sample 
 class 2 has 188 sample 


Now, we can do a groupby operation to check detailed information

In [44]:
df['cluster']=cluster_label
df.groupby('cluster').agg(Sum_default=('default payment next month', 'sum'), Count=('default payment next month', 'count'),Avg_age=('AGE', 'mean'),Avg_EDU=('EDUCATION', 'mean'),Avg_MARRIAGE=('MARRIAGE', 'mean'))


KeyError: "Column 'default payment next month' does not exist!"

We can see the 3-rd cluster customers have a huge pike on their PAY_AMT5, we want to make a further analysis on this group to see what happened！

Let do a groupby first to check gender, education, default distribution for this group.

In [ ]:
df['cluster_mark']=cluster_label
result=df.groupby('cluster_mark',as_index=0).agg(Sum_default=('default payment next month', 'sum'), Count=('default payment next month', 'count'),Avg_age=('AGE', 'mean'),Avg_EDU=('EDUCATION', 'mean'),Avg_MARRIAGE=('MARRIAGE', 'mean'))

In [ ]:
result['ratio']=result['Sum_default']/result['Count']
result

We have successfully trace back our customer's personal information from their bill&payment behaviors.

From line chart, you might have found that cluster 3 and cluster 4 customers' bill&payment behaviors are a little 'bumpy' or 'oscillating'. However, their default rate are the lowest.

What kind of information can be deducted from this plot? 

We might guess that customers inside these two clusters might take the loan for business. They need the loan to purchase, and they will pay back their loan once they receive their revenue.On the other hands, clusters 0,1,2,show a very smooth trend, which might indicate these bills type is house loan or other fixed bills.

What we can do for this analysis?

We could create new credit products for business loans to encourage them do more investment on their business as the rewards for their good credit!

Also, have you seen the education level for cluster 3 and 4? They are the lowest.So,can we recommend some online-MBA courses for these entrepreneurs?





In [43]:
len(df.columns)

19